# Overview of database files and file groups

In [82]:
SELECT
    db.name AS DBName,
    mf.type_desc AS FileType,
    ds.name AS FileGroup,
    Physical_Name AS Location
FROM
    sys.master_files mf
INNER JOIN sys.databases db ON db.database_id = mf.database_id
INNER JOIN sys.data_spaces ds ON ds.data_space_id = mf.data_space_id
WHERE db.name = 'WideWorldImporters'


(3 rows affected)

Total execution time: 00:00:00.058

DBName,FileType,FileGroup,Location
WideWorldImporters,ROWS,PRIMARY,C:\Program Files\Microsoft SQL Server\MSSQL13.MSSQLSERVER\MSSQL\DATA\WideWorldImporters.mdf
WideWorldImporters,ROWS,USERDATA,C:\Program Files\Microsoft SQL Server\MSSQL13.MSSQLSERVER\MSSQL\DATA\WideWorldImporters_UserData.ndf
WideWorldImporters,FILESTREAM,WWI_InMemory_Data,C:\Program Files\Microsoft SQL Server\MSSQL13.MSSQLSERVER\MSSQL\DATA\WideWorldImporters_InMemory_Data_1


# Delete Scheme and Function to start over

In [ ]:
IF EXISTS (SELECT * FROM sys.partition_schemes WHERE name = 'PS_OrderDate')  DROP PARTITION SCHEME [PS_OrderDate];
GO 
IF EXISTS (SELECT * FROM sys.partition_functions WHERE name = 'PF_OrderDate')  DROP PARTITION FUNCTION [PF_OrderDate];
GO

# Create Partition Function and Scheme

In [84]:
-- Create Partition Function dynamically from 2019-08-01 to three months ago
DECLARE @WorkingDate DATE;
DECLARE @DateString NVARCHAR(MAX);
DECLARE @PartitionCountPrimary INT;
DECLARE @PartitionCountSecondary INT;
DECLARE @sql_function NVARCHAR(max);

SET @WorkingDate = '20191001';
SET @DateString = N'';
SET @PartitionCountPrimary = 0;
SET @PartitionCountSecondary = 0;

-- Loop per month
WHILE @WorkingDate < DATEADD(MONTH, -2, CONVERT(DATE, FORMAT(GETDATE(), 'yyyy-MM-01'))) -- Set the number of months to create monthly partitions
BEGIN
    SET @DateString = @DateString + CONVERT(NVARCHAR(8), @WorkingDate, 112) + N', ';
	SET @PartitionCountSecondary = @PartitionCountSecondary + 1;
    SET @WorkingDate = DATEADD(MONTH, 1, @WorkingDate);
END;

-- Loop per day from the last three months
WHILE @WorkingDate <= CONVERT(DATE, GETDATE()) -- Loop through single days
BEGIN
    SET @DateString = @DateString + CONVERT(NVARCHAR(8), @WorkingDate, 112) + N', ';
	SET @PartitionCountPrimary = @PartitionCountPrimary + 1;
    SET @WorkingDate = DATEADD(DAY, 1, @WorkingDate);
END;

SET @DateString = @DateString + CONVERT(NVARCHAR(8), @WorkingDate, 112);
SET @PartitionCountPrimary = @PartitionCountPrimary + 1;

SET @sql_function = 'CREATE PARTITION FUNCTION [PF_OrderDate] (INT) AS RANGE RIGHT FOR VALUES (' + @DateString + ')'

PRINT '--> The following script is executed to create the partition function, it defines the boundaries of the partition'
PRINT @sql_function
EXEC sp_executesql @sql_function;

-- Create Partition Scheme dynamically based on the Partition Function (canr be replaced by "ALL TO" when all partitions need to be in one filegroup!)
DECLARE @sql_scheme NVARCHAR(MAX);
DECLARE @loopCount INT = 0;

SET @sql_scheme = 'CREATE PARTITION SCHEME PS_OrderDate AS PARTITION [PF_OrderDate] TO ('

WHILE @loopCount < @PartitionCountSecondary
BEGIN
	
	SET @sql_scheme = @sql_scheme + '[USERDATA], ';
	
	SET @loopCount = @loopCount + 1;
END
SET @sql_scheme = @sql_scheme + '[USERDATA], ' -- Last one without a trailing comma

SET @loopCount = 0;
WHILE @loopCount < @PartitionCountPrimary
BEGIN
	
	SET @sql_scheme = @sql_scheme + '[PRIMARY], ';
	
	SET @loopCount = @loopCount + 1;
END
SET @sql_scheme = @sql_scheme + '[PRIMARY]);' -- Last one without a trailing comma

PRINT '--> The following script is executed to create the partition scheme, it defines on which filegroups the partitions are located'
PRINT @sql_scheme

EXEC sp_executesql @sql_scheme;
GO

--> The following script is executed to create the partition function, it defines the boundaries of the partition

CREATE PARTITION FUNCTION [PF_OrderDate] (INT) AS RANGE RIGHT FOR VALUES (20191001, 20191101, 20191201, 20200101, 20200201, 20200301, 20200401, 20200501, 20200601, 20200701, 20200801, 20200901, 20201001, 20201101, 20201201, 20210101, 20210201, 20210301, 20210401, 20210402, 20210403, 20210404, 20210405, 20210406, 20210407, 20210408, 20210409, 20210410, 20210411, 20210412, 20210413, 20210414, 20210415, 20210416, 20210417, 20210418, 20210419, 20210420, 20210421, 20210422, 20210423, 20210424, 20210425, 20210426, 20210427, 20210428, 20210429, 20210430, 20210501, 20210502, 20210503, 20210504, 20210505, 20210506, 20210507, 20210508, 20210509, 20210510, 20210511, 20210512, 20210513, 20210514, 20210515, 20210516, 20210517, 20210518, 20210519, 20210520, 20210521, 20210522, 20210523, 20210524, 20210525, 20210526, 20210527, 20210528, 20210529, 20210530, 20210531, 20210601, 20210602, 20210603, 20210604, 20210605, 20210606, 20210607, 20210608, 20210609, 20210610, 20210611, 20210612, 20210613)

: Msg 2714, Level 16, State 58, Line 1
In der Datenbank ist bereits ein Objekt mit dem Namen "PF_OrderDate" vorhanden.

Total execution time: 00:00:00.007

# Delete and Create Clustered Index on partition-column

In [85]:
IF EXISTS (SELECT * FROM sys.indexes WHERE name = 'IX_Sales_Orders_OrderDate') DROP INDEX IX_Sales_Orders_OrderDate ON Sales.Orders_Part;
GO
CREATE CLUSTERED INDEX IX_Sales_Orders_OrderDate
  ON Sales.Orders_Part (OrderDate_int)
  WITH (STATISTICS_NORECOMPUTE = OFF, IGNORE_DUP_KEY = OFF,
         ALLOW_ROW_LOCKS = ON, ALLOW_PAGE_LOCKS = ON)
  ON PS_OrderDate(OrderDate_int)

Commands completed successfully.

Commands completed successfully.

Total execution time: 00:00:09.510

### Use this code to unpartition (to "reset" the test)

In [ ]:
CREATE CLUSTERED INDEX IX_Sales_Orders_OrderDate
    ON Sales.Orders_Part (OrderDate_int)
    WITH (DROP_EXISTING = ON)
    ON [PRIMARY];
GO

# Step 1: A different table on the slow partition needs to be created to move all data from three months ago into one partition

## Step 1a: Create table

In [86]:
DROP TABLE IF EXISTS [Sales].[Orders_Part_Merge];
CREATE TABLE [Sales].[Orders_Part_Merge](
	[OrderID] [int] NOT NULL,
	[CustomerID] [int] NOT NULL,
	[SalespersonPersonID] [int] NOT NULL,
	[PickedByPersonID] [int] NULL,
	[ContactPersonID] [int] NOT NULL,
	[BackorderOrderID] [int] NULL,
	[OrderDate] [date] NOT NULL,
	[OrderDate_int] [int] NOT NULL,
	[ExpectedDeliveryDate] [date] NOT NULL,
	[CustomerPurchaseOrderNumber] [nvarchar](20) NULL,
	[IsUndersupplyBackordered] [bit] NOT NULL,
	[Comments] [nvarchar](max) NULL,
	[DeliveryInstructions] [nvarchar](max) NULL,
	[InternalComments] [nvarchar](max) NULL,
	[PickingCompletedWhen] [datetime2](7) NULL,
	[LastEditedBy] [int] NOT NULL,
	[LastEditedWhen] [datetime2](7) NOT NULL
)
ON [USERDATA]
GO

Commands completed successfully.

Total execution time: 00:00:00.007

## Step 1b: Move data from daily partitions into the other table

In [87]:
DECLARE @RangeFrom DATE;
DECLARE @RangeTo DATE;
DECLARE @WorkingDate DATE;

SET @RangeFrom = (SELECT TOP(1) CONVERT(DATE, FORMAT(DATEADD(MONTH, -2, GETDATE()), 'yyyy-MM') + '-01'));
SET @RangeTo = DATEADD(MONTH, 1, @RangeFrom);
SET @WorkingDate = @RangeFrom;

PRINT 'Range from: ' + CONVERT(varchar(10), @RangeFrom);
PRINT 'Range to: ' + CONVERT(varchar(10), @RangeTo);

WHILE @WorkingDate < @RangeTo
BEGIN
	
	-- Insert into the table
	INSERT INTO	[Sales].[Orders_Part_Merge] WITH (TABLOCKX)
    (
       [OrderID]
      ,[CustomerID]
      ,[SalespersonPersonID]
      ,[PickedByPersonID]
      ,[ContactPersonID]
      ,[BackorderOrderID]
      ,[OrderDate]
      ,[OrderDate_int]
      ,[ExpectedDeliveryDate]
      ,[CustomerPurchaseOrderNumber]
      ,[IsUndersupplyBackordered]
      ,[Comments]
      ,[DeliveryInstructions]
      ,[InternalComments]
      ,[PickingCompletedWhen]
      ,[LastEditedBy]
      ,[LastEditedWhen]
    )

	SELECT [OrderID]
      ,[CustomerID]
      ,[SalespersonPersonID]
      ,[PickedByPersonID]
      ,[ContactPersonID]
      ,[BackorderOrderID]
      ,[OrderDate]
      ,[OrderDate_int]
      ,[ExpectedDeliveryDate]
      ,[CustomerPurchaseOrderNumber]
      ,[IsUndersupplyBackordered]
      ,[Comments]
      ,[DeliveryInstructions]
      ,[InternalComments]
      ,[PickingCompletedWhen]
      ,[LastEditedBy]
      ,[LastEditedWhen] FROM [Sales].[Orders_Part] 
	WHERE CONVERT(DATE, [OrderDate]) = @WorkingDate

    PRINT 'Inserted for ' + CONVERT(varchar(10), @WorkingDate);

	SET @WorkingDate = DATEADD(DAY, 1, @WorkingDate);
END
GO

Range from: 2021-04-01

Range to: 2021-05-01

(1431 rows affected)

Inserted for 2021-04-01

(1412 rows affected)

...

(1459 rows affected)

Inserted for 2021-04-29

(1441 rows affected)

Inserted for 2021-04-30

Total execution time: 00:00:05.051

## Step 1c: Check content of the Merge table

In [88]:
SELECT OrderDate, count(*) AS CountRows
FROM [Sales].[Orders_Part_Merge]
GROUP BY OrderDate
ORDER BY OrderDate

(30 rows affected)

Total execution time: 00:00:00.078

OrderDate,CountRows
2021-04-01,1431
2021-04-02,1412
2021-04-03,1475
2021-04-04,1439
2021-04-05,1405
2021-04-06,1484
2021-04-07,1443
2021-04-08,1497
2021-04-09,1430
2021-04-10,1460


# Step 2: Merge partition function

In [89]:
DECLARE @RangeFrom DATE;
DECLARE @RangeTo DATE;
DECLARE @WorkingDate DATE;

SET @RangeFrom = (SELECT TOP(1) CONVERT(DATE, FORMAT(DATEADD(MONTH, -2, GETDATE()), 'yyyy-MM') + '-02'));
SET @RangeTo = DATEADD(DAY, -1, DATEADD(MONTH, 1, @RangeFrom));
SET @WorkingDate = @RangeFrom;

PRINT 'Range from: ' + CONVERT(varchar(10), @RangeFrom);
PRINT 'Range to: ' + CONVERT(varchar(10), @RangeTo);

-- Merge partitions
WHILE @WorkingDate < @RangeTo
BEGIN

	--PRINT @WorkingDate
	ALTER PARTITION FUNCTION PF_OrderDate() MERGE RANGE (CONVERT(INT, CONVERT(nvarchar(8), @WorkingDate, 112)));

	SET @WorkingDate = DATEADD(DAY, 1, @WorkingDate);

END
GO

--> Check Partition overview, the partition should be merged but still located on the PRIMARY partition

Range from: 2021-04-02

Range to: 2021-05-01

Commands completed successfully.

Total execution time: 00:00:00.874

# Step 3: Delete from fast storage through a temp-table

## Step 3a: Create temp-table on PRIMARY

In [90]:
DROP TABLE IF EXISTS [Sales].[Orders_Part_Temp];
CREATE TABLE [Sales].[Orders_Part_Temp](
	[OrderID] [int] NOT NULL,
	[CustomerID] [int] NOT NULL,
	[SalespersonPersonID] [int] NOT NULL,
	[PickedByPersonID] [int] NULL,
	[ContactPersonID] [int] NOT NULL,
	[BackorderOrderID] [int] NULL,
	[OrderDate] [date] NOT NULL,
	[OrderDate_int] [int] NOT NULL,
	[ExpectedDeliveryDate] [date] NOT NULL,
	[CustomerPurchaseOrderNumber] [nvarchar](20) NULL,
	[IsUndersupplyBackordered] [bit] NOT NULL,
	[Comments] [nvarchar](max) NULL,
	[DeliveryInstructions] [nvarchar](max) NULL,
	[InternalComments] [nvarchar](max) NULL,
	[PickingCompletedWhen] [datetime2](7) NULL,
	[LastEditedBy] [int] NOT NULL,
	[LastEditedWhen] [datetime2](7) NOT NULL
)
ON [PRIMARY]
GO
-- Clustered index should not be partitioned!!
CREATE CLUSTERED INDEX IX_Sales_Orders_Temp_OrderDate ON Sales.Orders_Part_Temp (OrderDate_int)
	WITH (STATISTICS_NORECOMPUTE = OFF, IGNORE_DUP_KEY = OFF,
			ALLOW_ROW_LOCKS = ON, ALLOW_PAGE_LOCKS = ON);


Commands completed successfully.

Commands completed successfully.

Total execution time: 00:00:00.013

## Step 3b: Switch partition to temp-table

In [91]:
DECLARE @PartitionId INT;
DECLARE @RightBoundary INT;

SET @RightBoundary = (SELECT TOP(1) CONVERT(int, CONVERT(nvarchar(8), DATEADD(month, DATEDIFF(month, 0, DATEADD(MONTH, -1, GETDATE())), 0), 112)));

SET @PartitionId = (SELECT TOP (1) p.partition_number
					FROM sys.partition_functions AS pf
					JOIN sys.partition_schemes AS ps ON ps.function_id = pf.function_id
					JOIN sys.indexes AS si ON si.data_space_id = ps.data_space_id
					JOIN sys.partitions AS p ON  si.object_id = p.object_id AND si.index_id = p.index_id
					LEFT JOIN sys.partition_range_values rv ON  pf.function_id = rv.function_id
															AND p.partition_number = rv.boundary_id
					WHERE pf.name = 'PF_OrderDate'
					AND rv.value = @RightBoundary)

PRINT 'Switching Partition ID  ' + CONVERT(varchar(10), @PartitionId); 

ALTER TABLE [Sales].[Orders_Part] SWITCH PARTITION @PartitionId TO [Sales].[Orders_Part_Temp];

-- Check Partition overview. The partition should still be there, but without any rows

Switching Partition ID 20

Total execution time: 00:00:00.078

## Step 3c: Truncate temp-table

In [92]:
TRUNCATE TABLE [Sales].[Orders_Part_Temp];
GO

Commands completed successfully.

Total execution time: 00:00:00.004

# Step 4: Move the partition from two months ago to the SECONDARY filegroup

In [93]:
DECLARE @RightBoundary INT;
SET @RightBoundary = (SELECT TOP(1) CONVERT(int, CONVERT(nvarchar(8), DATEADD(month, DATEDIFF(month, 0, DATEADD(MONTH, -2, GETDATE())), 0), 112)));

print @RightBoundary

ALTER PARTITION FUNCTION PF_OrderDate() MERGE RANGE(@RightBoundary);

ALTER PARTITION SCHEME PS_OrderDate NEXT USED [USERDATA];

ALTER PARTITION FUNCTION PF_OrderDate() SPLIT RANGE(@RightBoundary);
GO

-- Check Partition overview. The only change is that the partition has moved from PRIMARY to the secondary partition

20210401

Commands completed successfully.

Total execution time: 00:00:00.041

# Step 5: Move the data from the Merge-table into the main table (on SECONDARY, monthly partition)

## Step 5a: Put a clustered index on the Merge-table in order to switch

In [94]:
CREATE CLUSTERED INDEX IX_Sales_Orders_Part_Merge_OrderDate
  ON Sales.Orders_Part_Merge (OrderDate_int)
  WITH (STATISTICS_NORECOMPUTE = OFF, IGNORE_DUP_KEY = OFF,
         ALLOW_ROW_LOCKS = ON, ALLOW_PAGE_LOCKS = ON)
  ON PS_OrderDate(OrderDate_int)

Commands completed successfully.

Total execution time: 00:00:00.619

## Step 5b: Switch the partition from the merge table to the main table

In [95]:
DECLARE @PartitionId INT;
DECLARE @RightBoundary INT;

SET @RightBoundary = (SELECT TOP(1) CONVERT(int, CONVERT(nvarchar(8), DATEADD(month, DATEDIFF(month, 0, DATEADD(MONTH, -1, GETDATE())), 0), 112)));

SET @PartitionId = (SELECT TOP (1) p.partition_number
					FROM sys.partition_functions AS pf
					JOIN sys.partition_schemes AS ps ON ps.function_id = pf.function_id
					JOIN sys.indexes AS si ON si.data_space_id = ps.data_space_id
					JOIN sys.partitions AS p ON  si.object_id = p.object_id AND si.index_id = p.index_id
					LEFT JOIN sys.partition_range_values rv ON  pf.function_id = rv.function_id
															AND p.partition_number = rv.boundary_id
					WHERE pf.name = 'PF_OrderDate'
					AND rv.value = @RightBoundary)

PRINT 'Switching Partition ID  ' + CONVERT(varchar(10), @PartitionId); 

ALTER TABLE [Sales].[Orders_Part_Merge] SWITCH PARTITION @PartitionId TO [Sales].[Orders_Part] PARTITION @PartitionId;

-- Check Partition overview. The end result has been reached, the complete month on one partition in the secondary filegroup! :)

Switching Partition ID 20

Total execution time: 00:00:00.123

# Cleanup

In [ ]:
DROP TABLE IF EXISTS [Sales].[Orders_Part_Merge];
DROP TABLE IF EXISTS [Sales].[Orders_Part_Temp];

-- Unpartition table
CREATE CLUSTERED INDEX IX_Sales_Orders_OrderDate
    ON Sales.Orders_Part (OrderDate_int)
    WITH (DROP_EXISTING = ON)
    ON [PRIMARY];
GO